In [1]:
# -*- coding: utf-8 -*-
import json
import pandas as pd
import time
import pprint

# --- 步驟 1: 讀取 id_map 和 value_maps ---

# 讀取 id_map (變項代碼 -> 變項說明)
id_map_file_path = 'tigps_w1_id_map.json' # 假設您已儲存 id_map
loaded_id_map = None
try:
    with open(id_map_file_path, 'r', encoding='utf-8') as f:
        loaded_id_map = json.load(f)
    print(f"成功從 {id_map_file_path} 讀取 id_map。")
except FileNotFoundError:
    print(f"錯誤：找不到檔案 {id_map_file_path}。請確認檔案路徑和名稱。")
    exit()
except Exception as e:
    print(f"讀取 id_map 時發生錯誤：{e}")
    exit()

# 讀取 value_maps (變項代碼 -> {選項代碼: 選項說明})
# *** 請確保這個 JSON 檔案是您更新過的、包含正確 as70a 等主要活動映射的版本 ***
value_map_file_path = 'tigps_w1_value_maps.json'
loaded_value_maps = None
try:
    with open(value_map_file_path, 'r', encoding='utf-8') as f:
        loaded_value_maps = json.load(f)
    print(f"成功從 {value_map_file_path} 讀取 value_maps。")
except FileNotFoundError:
    print(f"錯誤：找不到檔案 {value_map_file_path}。")
    exit()
except Exception as e:
    print(f"讀取 value_maps 時發生錯誤：{e}")
    exit()

# --- 步驟 2: 處理 value_maps 的鍵類型 (字串轉回整數) ---
print("正在處理 value map 鍵類型 (字串轉整數)...")
processed_value_maps = {}
if loaded_value_maps:
    for var_name, inner_map in loaded_value_maps.items():
        if isinstance(inner_map, dict):
            new_inner_map = {}
            for key_str, value in inner_map.items():
                try:
                    key_int = int(key_str)
                    new_inner_map[key_int] = value
                except ValueError:
                    new_inner_map[key_str] = value
            processed_value_maps[var_name] = new_inner_map
        else:
            processed_value_maps[var_name] = inner_map
    print("處理 value map 鍵類型完成。")
else:
    print("錯誤：未能成功載入 value_maps。")
    exit()

# --- 步驟 3: 載入您的實際完整資料 ---
csv_file_path = '../data/TIGPSw1_s.csv'
raw_data_df = None
print(f"\n正在從 {csv_file_path} 載入完整資料...")
start_time = time.time()
try:
    raw_data_df = pd.read_csv(csv_file_path) #, low_memory=False)
    end_time = time.time()
    print(f"成功載入資料。耗時: {end_time - start_time:.2f} 秒。")
    print(f"原始資料維度 (行數, 欄數): {raw_data_df.shape}")
except FileNotFoundError:
    print(f"錯誤：找不到您的資料檔案 {csv_file_path}。")
    exit()
except Exception as e:
    print(f"載入 CSV 檔案時發生錯誤：{e}")
    exit()

# --- 步驟 4: 複製原始資料並進行欄位名稱轉換 ---
print("\n--- 正在複製資料並轉換欄位名稱 ---")
# 為了安全，只對 raw_data_df 中存在的欄位進行 rename
valid_rename_map = {k: v for k, v in loaded_id_map.items() if k in raw_data_df.columns}
# 執行 rename
descriptive_df = raw_data_df.rename(columns=valid_rename_map)
print(f"已將 {len(valid_rename_map)} 個欄位名稱從代碼轉換為說明。")
print(f"描述性欄位名稱 DataFrame 維度: {descriptive_df.shape}")

# 建立反向映射，方便後面查找原始代碼
# inverse_id_map = {v: k for k, v in valid_rename_map.items()} # 直接反轉可能因重複說明而出錯
# 更安全的方式：遍歷 valid_rename_map
original_code_from_desc = {}
for code, desc in valid_rename_map.items():
    if desc not in original_code_from_desc: # 只保留第一個遇到的代碼（如果說明有重複）
        original_code_from_desc[desc] = code


# --- 步驟 5: 在 descriptive_df 上進行值的轉換 (選項代碼 -> 選項說明) ---
print("\n--- 開始在描述性欄位名稱的 DataFrame 上轉換數值為標籤 ---")
# 注意：這次我們是直接修改 descriptive_df 的欄位值
start_time = time.time()
value_mapped_count = 0
value_skipped_count = 0

# 遍歷 descriptive_df 的欄位名稱 (這些是中文說明)
for desc_col_name in descriptive_df.columns:
    # 透過反向查找，找到這個中文說明對應的原始變項代碼
    original_code = original_code_from_desc.get(desc_col_name)

    # 如果找到了原始代碼，並且該代碼存在於 processed_value_maps 中
    if original_code and original_code in processed_value_maps:
        value_map = processed_value_maps[original_code]
        if isinstance(value_map, dict) and value_map:
            # 應用 map，直接覆蓋原始數值
            # 使用 apply/lambda 可能比 map 更能處理潛在的 dtype 問題
            # descriptive_df[desc_col_name] = descriptive_df[desc_col_name].map(value_map)
            # 為了避免因混合類型 (數字和 NaN/字串) 導致 map 失效，用 replace 更穩健
            # 先檢查原始欄位的類型，如果是 object (可能含字串)，先嘗試轉數字
            if pd.api.types.is_object_dtype(descriptive_df[desc_col_name]):
                 # errors='coerce' 會將無法轉換的值變為 NaT/NaN/None
                 descriptive_df[desc_col_name] = pd.to_numeric(descriptive_df[desc_col_name], errors='coerce')

            # 使用 replace 進行轉換，未在 map 中的值保持原樣 (或者變成 NaN，取決於原始值)
            descriptive_df[desc_col_name] = descriptive_df[desc_col_name].replace(value_map)

            value_mapped_count += 1
            # print(f"  已轉換值: '{desc_col_name}' (原始代碼: {original_code})")
        else:
            value_skipped_count +=1 # value map 無效
    else:
        # 這個欄位可能是 ID、文字輸入欄位，或者不在 id_map/value_maps 中
        value_skipped_count += 1

end_time = time.time()
print("\n--- 值轉換完成 ---")
print(f"成功轉換 {value_mapped_count} 個欄位的值。")
print(f"跳過 {value_skipped_count} 個欄位的值 (可能原因：非選項代碼欄位、無有效 value map)。")
print(f"值轉換過程耗時: {end_time - start_time:.2f} 秒。")
print(f"最終 DataFrame 維度: {descriptive_df.shape}")

# --- 步驟 6: 查看最終結果 (範例) ---
print("\n--- 最終結果預覽 (欄位名稱和值都已轉換，前 5 筆) ---")
pd.set_option('display.max_columns', 20) # 限制顯示欄位數，避免過寬
pd.set_option('display.max_colwidth', 50) # 限制欄位寬度
print(descriptive_df.head())

# --- 步驟 7: (可選) 儲存最終結果 ---
final_output_path = 'TIGPSw1_s_descriptive_labeled.csv'
print(f"\n--- 正在將最終結果儲存至 {final_output_path} ---")
start_time = time.time()
try:
    descriptive_df.to_csv(final_output_path, index=False, encoding='utf-8-sig')
    end_time = time.time()
    print(f"成功儲存檔案。耗時: {end_time - start_time:.2f} 秒。")
except Exception as e:
    print(f"儲存最終結果 CSV 檔案時發生錯誤：{e}")

成功從 tigps_w1_id_map.json 讀取 id_map。
成功從 tigps_w1_value_maps.json 讀取 value_maps。
正在處理 value map 鍵類型 (字串轉整數)...
處理 value map 鍵類型完成。

正在從 ../data/TIGPSw1_s.csv 載入完整資料...


/var/folders/70/_kqz373j67x45fzvlzzjnn300000gq/T/ipykernel_12773/3731992145.py:65: DtypeWarning: Columns (19,29,152,324,340,368,383,481,483,505,507,529,531,553,555,577,579,603,627) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data_df = pd.read_csv(csv_file_path) #, low_memory=False)


成功載入資料。耗時: 2.29 秒。
原始資料維度 (行數, 欄數): (8958, 688)

--- 正在複製資料並轉換欄位名稱 ---
已將 688 個欄位名稱從代碼轉換為說明。
描述性欄位名稱 DataFrame 維度: (8958, 688)

--- 開始在描述性欄位名稱的 DataFrame 上轉換數值為標籤 ---

--- 值轉換完成 ---
成功轉換 590 個欄位的值。
跳過 98 個欄位的值 (可能原因：非選項代碼欄位、無有效 value map)。
值轉換過程耗時: 15.43 秒。
最終 DataFrame 維度: (8958, 688)

--- 最終結果預覽 (欄位名稱和值都已轉換，前 5 筆) ---
   題本編號               問卷名稱        學生 ID  學校 ID  班級  學校班級 ID  學校類型  座號  \
0  CO01  2023國中段正式施測-學生問卷A  s0020100001      2   1      201     2   1   
1  CO01  2023國中段正式施測-學生問卷A  s0020100002      2   1      201     2   2   
2  CO01  2023國中段正式施測-學生問卷A  s0020100003      2   1      201     2   3   
3  CO01  2023國中段正式施測-學生問卷A  s0020100004      2   1      201     2   5   
4  CO01  2023國中段正式施測-學生問卷A  s0020100005      2   1      201     2   6   

  請問你的性別(生理性別)?  請問你是哪一年出生?  ... 你開始長鬍子了嗎?(男生回答,女生請填1)  \
0             男          99  ...                只長出一點點   
1             男          99  ...                只長出一點點   
2             男          99  ...                只長出一點點   
3      